# Non-time series baseline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

new_origin = 'drive/MyDrive/CS229B_data/'
os.chdir(new_origin)

In [ ]:
!pip install flaml
!pip install hierarchicalforecast

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
import lightgbm as lgb
from flaml import AutoML
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MinTrace
from sklearn.metrics import mean_squared_error, mean_absolute_error
import gc
sns.set_theme(style="darkgrid")
import joblib

# Data preprocessing and feature engineering

In [23]:
load_df = pd.read_csv('data/load.csv')
hierarchy_df = pd.read_csv("data/hierarchy.csv")
humidity_df = pd.read_csv("data/relative humidity.csv")
temperature_df = pd.read_csv("data/temperature.csv")

In [26]:
def df_formatting(load_df, hierarchy_df, humidity_df, temperature_df):
    load_df = pd.melt(load_df, id_vars=["meter_id", "date"], value_vars=load_df.columns.difference(["meter_id", "date"]),
                                var_name="hour", value_name="load")
    load_df["hour"] = load_df["hour"].str.strip("h").astype(int) - 1
    load_df["timestamp"] = pd.to_datetime(load_df["date"] + " " + load_df["hour"].astype(str) + ":00:00", format="%m/%d/%Y %H:%M:%S")
    load_df["meter_id"] = load_df["meter_id"].astype(int)
    load_df = load_df.drop(columns=["date", "hour"])
    # Remove meter ids that appear in training but not in test and conversely
    aggregate_list = [('max_timestamp', 'max'), ('min_timestamp', 'min')]
    meters_df = load_df.groupby("meter_id")["timestamp"].agg(aggregate_list).reset_index()
    excluded_meters_df = meters_df[(meters_df["max_timestamp"]<dt.datetime(2011,1,1)) | (meters_df["min_timestamp"]>dt.datetime(2011,1,1))]
    excluded_meters = excluded_meters_df["meter_id"].to_list() + [236]
    load_df = load_df[~load_df["meter_id"].isin(excluded_meters)]
    # Ensure that all training points are present, being possibly 0
    date_rng = pd.date_range(start='2005-01-01', end='2010-12-31 23:00:00', freq='1H')
    all_combinations = pd.MultiIndex.from_product([load_df['meter_id'].unique(), date_rng], names=['meter_id', 'timestamp'])
    df_all_combinations = pd.DataFrame(index=all_combinations).reset_index()
    df_filled = pd.merge(df_all_combinations, load_df, on=['meter_id', 'timestamp'], how='left', suffixes=('_orig', '')).fillna(0)
    df_filled = pd.concat([df_filled, load_df[load_df["timestamp"]>=dt.datetime(2011,1,1)]])
    df_filled = df_filled.sort_values(by=["meter_id", "timestamp"]).reset_index(drop=True)
    data_df = df_filled.merge(hierarchy_df, on="meter_id", how="left")
    data_df[hierarchy_df.columns.difference(["meter_id"])] = data_df[hierarchy_df.columns.difference(["meter_id"])].astype(str)

    humidity_df["timestamp"] = pd.to_datetime(humidity_df["date"] + " " + (humidity_df["hr"] - 1).astype(str) + ":00:00", format="%d%b%Y %H:%M:%S")
    temperature_df["timestamp"] = pd.to_datetime(temperature_df["date"] + " " + (temperature_df["hr"] - 1).astype(str) + ":00:00", format="%d%b%Y %H:%M:%S")
    humidity_df = humidity_df.drop(columns=["date", "hr"])
    temperature_df = temperature_df.drop(columns=["date", "hr"])
    data_df = data_df.merge(humidity_df, on="timestamp", how="left")
    data_df = data_df.merge(temperature_df, on="timestamp", how="left")
    return data_df

joined_data_df = df_formatting(load_df, hierarchy_df, humidity_df, temperature_df)

In [27]:
del load_df
del hierarchy_df
del humidity_df
del temperature_df
gc.collect()

0

In [28]:
def get_datasets(df):
  aggregation_dict = {"load": "sum"}
  aggregation_dict.update({col: "first" for col in df.columns if col not in ["load", "timestamp"]})
  top_level_df = df.groupby("timestamp").agg(aggregation_dict).reset_index()
  aggregate_level_names = pd.unique(df["aggregate"])
  aggregate_levels = {}
  for agg_level in aggregate_level_names:
    agg_level_df = df[df["aggregate"] == agg_level].copy()
    agg_level_df = agg_level_df.groupby("timestamp").agg(aggregation_dict).reset_index()
    aggregate_levels[agg_level] = agg_level_df
  mid_level_names = pd.unique(df["mid_level"])
  mid_levels = {}
  bottom_levels = {}
  for mid_level in mid_level_names:
    mid_level_df = df[df["mid_level"] == mid_level].copy()
    bottom_levels[mid_level] = mid_level_df
    mid_level_df = mid_level_df.copy().groupby("timestamp").agg(aggregation_dict).reset_index()
    mid_levels[mid_level] = mid_level_df
  return top_level_df, aggregate_levels, mid_levels, bottom_levels

In [29]:
top_level_df, aggregate_levels, mid_levels, bottom_levels = get_datasets(joined_data_df)

In [30]:
def feature_engineering(df, encode_meter_id):
    # Basic feature engineering: indicator of day of the week, month, is_holiday (in MA), one-hot encoding of hierarchies and of the meter id
    df['year'] = df['timestamp'].dt.year
    df['cos_month'] = np.cos(2*np.pi*(df['timestamp'].dt.month/12))
    df['sin_month'] = np.sin(2*np.pi*(df['timestamp'].dt.month/12))
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    holidays_MA = holidays.US(years=range(2005, 2012), state="MA")
    df['is_holiday'] = df['timestamp'].dt.date.isin(holidays_MA.keys())
    df['cos_hour'] = np.cos(2*np.pi*(df['timestamp'].dt.hour/24))
    df['sin_hour'] = np.sin(2*np.pi*(df['timestamp'].dt.hour/24))
    #df['holiday_name'] = df.apply(lambda row: "None" if not df['is_holiday'] else holidays_MA[row["timestamp"].date()], axis=1): too slow, to be optimized
    df = df.drop(columns=["timestamp"])

    onehots = []
    if encode_meter_id:
      meter_onehot = pd.get_dummies(df['meter_id'], prefix="meter")
      onehots.append(meter_onehot)
    df = df.drop(columns=["meter_id", "mid_level", "aggregate"])
    dow_onehot = pd.get_dummies(df['day_of_week'], prefix="dow")
    df = df.drop(columns=["day_of_week"])
    features_df = pd.concat([df, dow_onehot] + onehots, axis=1)
    return features_df

In [31]:
top_level_df = feature_engineering(top_level_df, False)
for agg in aggregate_levels.keys():
  aggregate_levels[agg] = feature_engineering(aggregate_levels[agg], False)
for mid_level in mid_levels.keys():
  mid_levels[mid_level] = feature_engineering(mid_levels[mid_level], False)
for mid_level in bottom_levels.keys():
  bottom_levels[mid_level] = feature_engineering(bottom_levels[mid_level], True)

In [32]:
def train_test_split(df):
    train_df = df[df["year"] < 2011].fillna(0)
    test_df = df[df["year"] == 2011].dropna()
    return train_df, test_df
top_level_train, top_level_test = train_test_split(top_level_df)
aggregate_train = {}
aggregate_test = {}
mid_levels_train = {}
mid_levels_test = {}
bottom_levels_train = {}
bottom_levels_test = {}
for agg in aggregate_levels.keys():
  train_df, test_df = train_test_split(aggregate_levels[agg])
  aggregate_train[agg] = train_df
  aggregate_test[agg] = test_df
for mid_level in mid_levels.keys():
  train_df, test_df = train_test_split(mid_levels[mid_level])
  mid_levels_train[mid_level] = train_df
  mid_levels_test[mid_level] = test_df
for mid_level in bottom_levels.keys():
  train_df, test_df = train_test_split(bottom_levels[mid_level])
  bottom_levels_train[mid_level] = train_df
  bottom_levels_test[mid_level] = test_df

# Training models

In [ ]:
config = {
    "time_budget": 300,  # 5 minute time budget for each model
    "metric": 'mse',
    "task": 'regression',
    "estimator_list": ["lgbm"]
}
top_model = AutoML()
y_train = top_level_train["load"]
X_train = top_level_train.drop(columns="load")
top_model.fit(X_train, y_train, **config)
best_top_model = top_model.model
joblib.dump(best_top_model, 'top_model.joblib')

[flaml.automl.logger: 11-12 08:01:07] {1679} INFO - task = regression
[flaml.automl.logger: 11-12 08:01:07] {1690} INFO - Evaluation method: holdout
[flaml.automl.logger: 11-12 08:01:07] {1788} INFO - Minimizing error metric: mse
[flaml.automl.logger: 11-12 08:01:07] {1900} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl.logger: 11-12 08:01:07] {2218} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 11-12 08:01:07] {2344} INFO - Estimated sufficient time budget=3182s. Estimated necessary time budget=3s.
[flaml.automl.logger: 11-12 08:01:07] {2391} INFO -  at 1.5s,	estimator lgbm's best error=53440622675.5967,	best estimator lgbm's best error=53440622675.5967
[flaml.automl.logger: 11-12 08:01:07] {2218} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 11-12 08:01:07] {2391} INFO -  at 1.6s,	estimator lgbm's best error=53440622675.5967,	best estimator lgbm's best error=53440622675.5967
[flaml.automl.logger: 11-12 08:01:07] {2218} INFO - iterat

['top_model.joblib']

In [ ]:
for agg in aggregate_train.keys():
  agg_model = AutoML()
  y_train = aggregate_train[agg]["load"]
  X_train = aggregate_train[agg].drop(columns="load")
  agg_model.fit(X_train, y_train, **config)
  best_agg_model = agg_model.model
  joblib.dump(best_agg_model, 'agg_model_'+ agg +'.joblib')

[flaml.automl.logger: 11-12 04:47:59] {1679} INFO - task = regression
[flaml.automl.logger: 11-12 04:47:59] {1690} INFO - Evaluation method: holdout
[flaml.automl.logger: 11-12 04:47:59] {1788} INFO - Minimizing error metric: mse
[flaml.automl.logger: 11-12 04:47:59] {1900} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl.logger: 11-12 04:47:59] {2218} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 11-12 04:47:59] {2344} INFO - Estimated sufficient time budget=30810s. Estimated necessary time budget=31s.
[flaml.automl.logger: 11-12 04:47:59] {2391} INFO -  at 2.2s,	estimator lgbm's best error=1386739244.5075,	best estimator lgbm's best error=1386739244.5075
[flaml.automl.logger: 11-12 04:47:59] {2218} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 11-12 04:48:00] {2391} INFO -  at 2.6s,	estimator lgbm's best error=1386739244.5075,	best estimator lgbm's best error=1386739244.5075
[flaml.automl.logger: 11-12 04:48:00] {2218} INFO - iteratio

In [ ]:
for mid in mid_levels_train.keys():
  mid_model = AutoML()
  y_train = mid_levels_train[mid]["load"]
  X_train = mid_levels_train[mid].drop(columns="load")
  mid_model.fit(X_train, y_train, **config)
  best_mid_model = mid_model.model
  joblib.dump(best_mid_model, 'mid_model_'+ mid +'.joblib')

[flaml.automl.logger: 11-12 04:59:10] {1679} INFO - task = regression
[flaml.automl.logger: 11-12 04:59:10] {1690} INFO - Evaluation method: holdout
[flaml.automl.logger: 11-12 04:59:10] {1788} INFO - Minimizing error metric: mse
[flaml.automl.logger: 11-12 04:59:10] {1900} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl.logger: 11-12 04:59:10] {2218} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 11-12 04:59:10] {2344} INFO - Estimated sufficient time budget=2806s. Estimated necessary time budget=3s.
[flaml.automl.logger: 11-12 04:59:10] {2391} INFO -  at 1.5s,	estimator lgbm's best error=60146063.1509,	best estimator lgbm's best error=60146063.1509
[flaml.automl.logger: 11-12 04:59:10] {2218} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 11-12 04:59:10] {2391} INFO -  at 1.5s,	estimator lgbm's best error=60146063.1509,	best estimator lgbm's best error=60146063.1509
[flaml.automl.logger: 11-12 04:59:10] {2218} INFO - iteration 2, curre

In [ ]:
for bottom in bottom_levels_train.keys():
  bottom_model = AutoML()
  y_train = bottom_levels_train[bottom]["load"]
  X_train = bottom_levels_train[bottom].drop(columns="load")
  bottom_model.fit(X_train, y_train, **config)
  best_bottom_model = bottom_model.model
  joblib.dump(best_bottom_model, 'bottom_model_'+ bottom +'.joblib')

[flaml.automl.logger: 11-12 06:25:41] {1679} INFO - task = regression
[flaml.automl.logger: 11-12 06:25:41] {1690} INFO - Evaluation method: holdout
[flaml.automl.logger: 11-12 06:25:43] {1788} INFO - Minimizing error metric: mse
[flaml.automl.logger: 11-12 06:25:43] {1900} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl.logger: 11-12 06:25:43] {2218} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 11-12 06:25:43] {2344} INFO - Estimated sufficient time budget=36836s. Estimated necessary time budget=37s.
[flaml.automl.logger: 11-12 06:25:43] {2391} INFO -  at 24.4s,	estimator lgbm's best error=2921788.2350,	best estimator lgbm's best error=2921788.2350
[flaml.automl.logger: 11-12 06:25:43] {2218} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 11-12 06:25:43] {2391} INFO -  at 24.5s,	estimator lgbm's best error=2921788.2350,	best estimator lgbm's best error=2921788.2350
[flaml.automl.logger: 11-12 06:25:43] {2218} INFO - iteration 2, curre

# Evaluating models and hierarchical reconciliation

In [ ]:
results_df = pd.DataFrame(columns=["level", "name", "rmse", "mae"])

# Top level
top_model = joblib.load("models/top_model.joblib")
y_test = top_level_test["load"]
X_test = top_level_test.drop(columns="load")
y_pred = top_model.predict(y_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
results_df.loc[len(results_df.index)] =["top", "top", rmse, mae]

# Aggregate level
for agg in aggregate_test.keys():
  agg_model = joblib.load('models/agg_model_'+ agg +'.joblib')
  y_test = aggregate_test[agg]["load"]
  X_test = aggregate_test[agg].drop(columns="load")
  y_pred = agg_model.predict(X_test)
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  mae = mean_absolute_error(y_test, y_pred)
  results_df.loc[len(results_df.index)] =["agg", agg, rmse, mae]

# Mid level
for mid in mid_levels_test.keys():
  mid_model = joblib.load('models/mid_model_'+ mid +'.joblib')
  y_test = mid_levels_test[mid]["load"]
  X_test = mid_levels_test[mid].drop(columns="load")
  y_pred = mid_model.predict(X_test)
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  mae = mean_absolute_error(y_test, y_pred)
  results_df.loc[len(results_df.index)] =["mid", mid, rmse, mae]

# Bottom level
for mid in bottom_levels_test.keys():
  mid_model = joblib.load('models/bottom_model_'+ mid +'.joblib')
  y_test = bottom_levels_test[mid]["load"]
  X_test = bottom_levels_test[mid].drop(columns="load")
  y_pred = mid_model.predict(X_test)
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  mae = mean_absolute_error(y_test, y_pred)
  results_df.loc[len(results_df.index)] =["bottom", mid, rmse, mae]

results_df.to_csv("results.csv")

In [46]:
# Encode the hierarchy information
meter_ids = pd.unique(joined_data_df["meter_id"])
mid_level_ids = pd.unique(joined_data_df["mid_level"])
aggregate_level_ids = pd.unique(joined_data_df["aggregate"])
n_series = 1 + len(aggregate_level_ids) + len(mid_level_ids) + len(meter_ids)
S = np.zeros((n_series, len(meter_ids)))
hierarchy_df = pd.read_csv("data/hierarchy.csv")
hierarchy_df = hierarchy_df[hierarchy_df["meter_id"].isin(meter_ids)]
hierarchy_df["indicator"] = 1
aggregate_indicators = hierarchy_df.pivot_table(index="aggregate", columns="meter_id", values="indicator", fill_value=0)
mid_level_indicators = hierarchy_df.pivot_table(index="mid_level", columns="meter_id", values="indicator", fill_value=0)

# rows / columns
row_names = ['Total'] + list(aggregate_level_ids) + list(mid_level_ids) + list(meter_ids)
S = pd.DataFrame(S, index=row_names, columns=meter_ids)

S.loc['Total', :] = 1
for agg in aggregate_level_ids:
  S.loc[agg] = aggregate_indicators.loc[agg]
for mid in mid_level_ids:
  S.loc[mid] = mid_level_indicators.loc[mid]
for meter_id in meter_ids:
  S.loc[meter_id, meter_id] = 1


tags = {}
tags['Total'] = np.array(['Total'], dtype=object)
tags['Total/Aggregate'] = aggregate_level_ids
tags['Total/Aggregate/MidLevel'] = mid_level_ids
tags['Total/Aggregate/MidLevel/Meter'] = meter_ids


In [60]:
h = len(top_level_test.index)
T = len(top_level_train.index)
y_hat = np.zeros((n_series, h))
Y_test = np.zeros((n_series, h))
Y_train = np.zeros((n_series, T))
Y_hat_train = np.zeros((n_series, T))
i_series = 0
mid_level_dict = hierarchy_df[["meter_id", "mid_level"]].set_index("meter_id").to_dict()["mid_level"]

top_model = joblib.load("models/top_model.joblib")
y_test = top_level_test["load"]
X_test = top_level_test.drop(columns="load")
y_pred = top_model.predict(X_test)
y_train = top_level_train["load"]
X_train = top_level_train.drop(columns="load")
y_hat_train = top_model.predict(X_train)
assert(len(y_train) == T)
assert(len(y_pred) == h)
y_hat[i_series, :] = y_pred.copy()
Y_test[i_series, :] = y_test.to_numpy()
Y_train[i_series, :] = y_train.to_numpy()
Y_hat_train[i_series, :] = y_hat_train.copy()
i_series += 1

# Aggregate level
for agg in aggregate_level_ids:
  agg_model = joblib.load('models/agg_model_'+ agg +'.joblib')
  y_test = aggregate_test[agg]["load"]
  X_test = aggregate_test[agg].drop(columns="load")
  y_pred = agg_model.predict(X_test)
  y_train = aggregate_train[agg]["load"]
  X_train = aggregate_train[agg].drop(columns="load")
  y_hat_train = top_model.predict(X_train)
  assert(len(y_train) == T)
  assert(len(y_pred) == h)
  y_hat[i_series, :] = y_pred.copy()
  Y_test[i_series, :] = y_test.to_numpy()
  Y_train[i_series, :] = y_train.to_numpy()
  Y_hat_train[i_series, :] = y_hat_train.copy()
  i_series += 1

# Mid level
for mid in mid_level_ids:
  mid_model = joblib.load('models/mid_model_'+ mid +'.joblib')
  y_test = mid_levels_test[mid]["load"]
  X_test = mid_levels_test[mid].drop(columns="load")
  y_pred = mid_model.predict(X_test)
  y_train = mid_levels_train[mid]["load"]
  X_train = mid_levels_train[mid].drop(columns="load")
  y_hat_train = mid_model.predict(X_train)
  assert(len(y_train) == T)
  assert(len(y_pred) == h)
  y_hat[i_series, :] = y_pred.copy()
  Y_test[i_series, :] = y_test.to_numpy()
  Y_train[i_series, :] = y_train.to_numpy()
  Y_hat_train[i_series, :] = y_hat_train.copy()
  i_series += 1

In [61]:
# Bottom level
for bottom in meter_ids:
  mid = mid_level_dict[bottom]
  mid_model = joblib.load('models/bottom_model_'+ mid +'.joblib')
  bottom_test = bottom_levels_test[mid][bottom_levels_test[mid]["meter_"+str(bottom)] == 1]
  bottom_train = bottom_levels_train[mid][bottom_levels_train[mid]["meter_"+str(bottom)] == 1]
  y_test = bottom_test["load"]
  X_test = bottom_test.drop(columns="load")
  if mid == 'E024':
    X_test["meter_236"] = 0
  y_pred = mid_model.predict(X_test)
  y_train = bottom_levels_train[mid][bottom_levels_train[mid]["meter_"+str(bottom)] == 1]["load"]
  X_train = bottom_train.drop(columns="load")
  if mid == 'E024':
    X_train["meter_236"] = 0
  y_hat_train = mid_model.predict(X_train)
  assert(len(y_train) == T)
  assert(len(y_pred) == h)
  y_hat[i_series, :] = y_pred.copy()
  Y_test[i_series, :] = y_test.to_numpy()
  Y_train[i_series, :] = y_train.to_numpy()
  Y_hat_train[i_series, :] = y_hat_train.copy()
  i_series += 1

In [62]:
y_hat_df = pd.DataFrame(y_hat, index=row_names, columns=[dt.datetime(2011,1,1)+i*dt.timedelta(seconds=3600) for i in range(h)])
y_hat_df = pd.melt(y_hat_df.reset_index().rename(columns={"index": "unique_id"}), id_vars="unique_id", var_name="ds", value_name="boost").set_index("unique_id")
Y_test = pd.DataFrame(Y_test, index=row_names, columns=[dt.datetime(2011,1,1)+i*dt.timedelta(seconds=3600) for i in range(h)])
Y_test = pd.melt(Y_test.reset_index().rename(columns={"index": "unique_id"}), id_vars="unique_id", var_name="ds", value_name="y").set_index("unique_id")
Y_train = pd.DataFrame(Y_train, index=row_names, columns=[dt.datetime(2005,1,1)+i*dt.timedelta(seconds=3600) for i in range(T)])
Y_train = pd.melt(Y_train.reset_index().rename(columns={"index": "unique_id"}), id_vars="unique_id", var_name="ds", value_name="y").set_index("unique_id")
Y_hat_train = pd.DataFrame(Y_hat_train, index=row_names, columns=[dt.datetime(2005,1,1)+i*dt.timedelta(seconds=3600) for i in range(T)])
Y_hat_train = pd.melt(Y_hat_train.reset_index().rename(columns={"index": "unique_id"}), id_vars="unique_id", var_name="ds", value_name="boost").set_index("unique_id")
Y_train = pd.concat([Y_train, Y_hat_train["boost"]], axis=1)

In [63]:
reconcilers = [MinTrace(method="ols"), MinTrace(method="wls_struct"), MinTrace(method="wls_var"), MinTrace(method="mint_shrink"), MinTrace(method="mint_cov"),  BottomUp(), TopDown(method="forecast_proportions")]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
y_hat_rec = hrec.reconcile(y_hat_df, S, tags, Y_train)

In [68]:
def rmse(x,y):
    return np.sqrt(mean_squared_error(x,y))
evaluator = HierarchicalEvaluation(evaluators=[rmse, mean_absolute_error])
evaluator.evaluate(Y_hat_df = y_hat_rec, Y_test_df = Y_test,
                   tags=tags)

boost  \
level                          metric                              
Overall                        rmse                  5287.747298   
                               mean_absolute_error   1300.349832   
Total                          rmse                 45997.709441   
                               mean_absolute_error  33866.501608   
Total/Aggregate                rmse                 30727.554131   
                               mean_absolute_error  19668.803576   
Total/Aggregate/MidLevel       rmse                  5752.520685   
                               mean_absolute_error   3079.704148   
Total/Aggregate/MidLevel/Meter rmse                  1679.400163   
                               mean_absolute_error     696.61792   

                                                   boost/MinTrace_method-ols  \
level                          metric                                          
Overall                        rmse                              5213.674509   
                               mean_absolute_error               1326.158168   
Total                          rmse                             45362.701701   
                               mean_absolute_error              33510.770837   
Total/Aggregate                rmse                             29987.053723   
                               mean_absolute_error              19384.951495   
Total/Aggregate/MidLevel       rmse                              5824.547542   
                               mean_absolute_error               3241.007063   
Total/Aggregate/MidLevel/Meter rmse                              1675.167006   
                               mean_absolute_error                715.938465   

                                                   boost/MinTrace_method-wls_struct  \
level                          metric                                                 
Overall                        rmse                                     5083.683967   
                               mean_absolute_error                      1278.880439   
Total                          rmse                                    44409.716699   
                               mean_absolute_error                     32810.854791   
Total/Aggregate                rmse                                     28864.87326   
                               mean_absolute_error                       18491.7208   
Total/Aggregate/MidLevel       rmse                                     5726.109946   
                               mean_absolute_error                      3084.285716   
Total/Aggregate/MidLevel/Meter rmse                                     1670.969992   
                               mean_absolute_error                       693.731167   

                                                   boost/MinTrace_method-wls_var  \
level                          metric                                              
Overall                        rmse                                  5066.645571   
                               mean_absolute_error                   1288.183725   
Total                          rmse                                 44247.775724   
                               mean_absolute_error                  32663.854061   
Total/Aggregate                rmse                                 28724.510619   
                               mean_absolute_error                  18430.153676   
Total/Aggregate/MidLevel       rmse                                  5756.852759   
                               mean_absolute_error                    3076.89426   
Total/Aggregate/MidLevel/Meter rmse                                  1660.809547   
                               mean_absolute_error                    706.483925   

                                                   boost/MinTrace_method-mint_shrink  \
level                          metric                                                  
Overall                        rmse                